# 第 6 章：信号处理与卷积数学（Signal Processing & Convolution）

本章核心目标：“吃透”卷积的数学本质——
- 不仅能看懂 Winograd/FFT 为什么能加速卷积，
- 还能在实际 Edge Runtime 里判断哪些卷积 kernel 应该走哪条加速路径（Direct vs FFT vs Winograd vs GEMM-Conv）。
这些数学为卷积加速提供理论基础。

---

## 6.1 卷积定理：时域卷积 = 频域乘法

### 6.1.1 一维离散卷积（数学定义）

$$
y[n] = (x * h)[n] = \sum_{k=-\infty}^{\infty} x[k]\,h[n-k]
$$

注意这里是$h[n-k]$，k 变大，h 的下标是 n − k，也就是在做卷积时，核是被翻转并平移的。
如果把索引换一下，会更清楚一点：令$m = n - k$，$k = n - m$代入：
$$y[n] = \sum_m x[n-m]\,h[m]$$
这其实是在做：

把 h 反向（翻转），然后

在 x 上滑动、做加权和

所以数学上：

卷积 = 卷积核翻转 + 平移 + 点乘 + 求和

### 6.1.2 深度学习里的 “卷积层”
深度学习框架（PyTorch / TensorFlow / cuDNN）里实现的通常是：$$y[n] = \sum_k x[n + k]\,w[k]$$
这叫 互相关（cross-correlation），kernel 根本没翻转，直接按原顺序滑过去。

所以严格来说，PyTorch 的 Conv1d / Conv2d 做的是相关（correlation），不是数学教科书里的卷积（convolution）。

关键点：参数是学出来的，不是固定的。
如果数学上真正的卷积是：
$$(x * h)[n] = \sum_k x[n-k]\,h[k]$$
而框架里实现的是：

$$\mathrm{corr}(x, w)[n] = \sum_k x[n+k]\,w[k]$$
两者的关系是：只要令 $w[k] = h[-k]$(也就是把 h 翻转一下), 那么：$$\mathrm{corr}(x, w) = x * h$$

也就是说：

如果你坚持做“数学意义上的卷积”，你会用 h_flip 这个核；

但在深度学习里，核是训练出来的，网络会自动学到一个“适合 correlation 实现”的权重张量；

总结：

- 数学上：卷积 = “翻转 + 乘 + 积分/求和”
- 深度学习：我们干脆把“翻转”这一部分也交给梯度下降去学，
- 所以实现用的是更简单的相关形式（不翻转核），也一样能表达同样的函数族。

所以大家就懒得说 “correlation layer” 了，统一叫 convolution layer，但底层实现是 correlation。

在实际 CNN 推理场景中，该公式用于：

- 单通道 1D conv（如 audio、时间序列）

- 2D 卷积的行/列展开

- 多维卷积（3D/temporal conv）的基础模块

### 6.1.3 卷积定理
DTFT 变换与卷积具有以下性质：
$\mathcal{F}\{x * h\} = X(\omega)H(\omega)$

> 卷积在频域里变成点乘。

计算复杂度：

- 直接卷积：$O(NK)$
- FFT-based：
  - $O(N \log N)$ 计算 FFT/逆 FFT
  - 中间是逐点乘法（$O(N)$）

离散卷积通过 FFT：
1. Zero-pad 输入与 kernel 使其卷积不 wrap-around
2. 执行 FFT：
$$X = \mathrm{FFT}(x),\quad H = \mathrm{FFT}(h)$$
3. 点乘：
$$Y = X \odot H$$
4. 执行 inverse FFT：
$$y = \mathrm{IFFT}(Y)$$
对 edge device runtime 什么时候 FFT 卷积更快？

当：

- 核 size 大（> 5×5）

- 输入 resolution 大（如视频、超分辨率、语音）

- 需要 group conv / depthwise conv 的 FFT 变体

- 做 runtime inference，会经常看到：

  - 对 7×7、11×11 conv 自动切换到 FFT kernel

  - 对 高分辨率摄像头帧 1600×1200，FFT 会非常划算

---


In [6]:
import torch
import torch.nn.functional as F

# input shape: (batch, channel, length)
x = torch.tensor([[ [0., 1., 2., 3., 4.] ]])    # (1, 1, 5)
w = torch.tensor([[ [-1., 0., 1.] ]])           # (1, 1, kernel_size=3)

# PyTorch Conv1d 实际做的是 cross-correlation
y_corr = F.conv1d(x, w, padding=0)  # shape (1, 1, 3)

# flip math kernel to get convolution
h_math = torch.flip(w, dims=[2])    # 在 kernel 维度翻转
y_conv_like = F.conv1d(x, h_math, padding=0)

print("PyTorch Conv1d:", y_corr)
print("math kernel flipped conv", y_conv_like)

PyTorch Conv1d: tensor([[[2., 2., 2.]]])
math kernel flipped conv tensor([[[-2., -2., -2.]]])



## 6.2 Toeplitz 与 Circulant 矩阵

### 6.2.1 Toeplitz 矩阵形状：卷积的矩阵写法
Toeplitz 矩阵（托普利茨矩阵）是一种“沿主对角线方向元素恒定”的特殊矩阵。
也就是说：矩阵的每一根从左上到右下的斜对角线上的所有元素都相等。

正式定义
一个 n×m 的矩阵 T 是 Toeplitz 矩阵，当且仅当满足：

$$T(i,j) = T(i+1,j+1) \quad \text{对所有合法的 } i,j$$
即矩阵元素只取决于行下标和列下标的差（或和）。

$$\begin{array}{c} T=
\begin{bmatrix}
h[0] & 0 & 0 & \cdots \\
h[1] & h[0] & 0 & \cdots \\
h[2] & h[1] & h[0] & \cdots\\
\vdots & & & \ddots
\end{bmatrix} \end{array}$$
特点：

- 每条对角线常数不变

- 卷积 → 一个特殊结构的稀疏矩阵与向量乘法


最常见的 Toeplitz 矩阵长这样（以一维卷积为例）：
$$
T = \begin{bmatrix}
t_{0}  & t_{1}  & t_{2}  & t_{3}  & t_{4}  \\[6pt]
t_{-1} & t_{0}  & t_{1}  & t_{2}  & t_{3}  \\[6pt]
t_{-2} & t_{-1} & t_{0}  & t_{1}  & t_{2}  \\[6pt]
t_{-3} & t_{-2} & t_{-1} & t_{0}  & t_{1}  \\[6pt]
t_{-4} & t_{-3} & t_{-2} & t_{-1} & t_{0}
\end{bmatrix}
$$

### 6.2.2 一维离散卷积和 Toeplitz 矩阵的关系

一维卷积可以写成 Toeplitz 矩阵与向量乘法：

$$
y = T x
$$

**例子**：
信号 $x = [x_0, x_1, x_2, x_3]$  $N=4$， 卷积核 h = $h = [h_0, h_1, h_2]$,长度= $K=3$
用数学意义上的一维离散卷积：
$$y[n] = \sum_{k=0}^{3} x[k]\, h[n-k],\quad n=0,\dots, N+K-2 = 5$$
边界之外（索引 <0 或 ≥长度）都视为 0（zero padding）。
输出长度为：
$$L = N + K - 1 = 4 + 3 - 1 = 6$$

也就是：$$y = [y_0, y_1, y_2, y_3, y_4, y_5]^\top$$

把每个$y_n$写成$x_0,\dots,x_3$的线性组合
- $n = 0$
$$\begin{aligned}
y_0 &= \sum_{k=0}^{3} x_k h[0-k] \\
&= x_0 h_0 + x_1 h[-1] + x_2 h[-2] + x_3 h[-3] \\
&= x_0 h_0 \quad (\text{因为 } h[-1],h[-2],h[-3] = 0)
\end{aligned}$$

- $n = 1$
$$\begin{aligned}
y_1 &= \sum_{k=0}^{3} x_k h[1-k] \\
&= x_0 h_1 + x_1 h_0 + x_2 h[-1] + x_3 h[-2] \\
&= x_0 h_1 + x_1 h_0
\end{aligned}$$

- $n = 2$
$$\begin{aligned}
y_2 &= \sum_{k=0}^{3} x_k h[2-k] \\
&= x_0 h_2 + x_1 h_1 + x_2 h_0 + x_3 h[-1] \\
&= x_0 h_2 + x_1 h_1 + x_2 h_0
\end{aligned}$$

- $n = 3$
$$\begin{aligned}
y_3 &= \sum_{k=0}^{3} x_k h[3-k] \\
&= x_0 h_3 + x_1 h_2 + x_2 h_1 + x_3 h_0 \\
&= x_1 h_2 + x_2 h_1 + x_3 h_0 \quad (\text{因为 } h_3 = 0)
\end{aligned}$$

- $n = 4$
$$\begin{aligned}
y_4 &= \sum_{k=0}^{3} x_k h[4-k] \\
&= x_0 h_4 + x_1 h_3 + x_2 h_2 + x_3 h_1 \\
&= x_2 h_2 + x_3 h_1
\end{aligned}$$

- $n = 5$
$$\begin{aligned}
y_5 &= \sum_{k=0}^{3} x_k h[5-k] \\
&= x_0 h_5 + x_1 h_4 + x_2 h_3 + x_3 h_2 \\
&= x_3 h_2
\end{aligned}$$
收集成矩阵形式$y = T x$
把$y_0,\dots,y_5$写成矩阵 × 向量形式：
$$\begin{bmatrix}
y_0\\
y_1\\
y_2\\
y_3\\
y_4\\
y_5
\end{bmatrix}
=
\begin{bmatrix}
h_0      & 0       & 0       & 0      \\
h_1      & h_0     & 0       & 0      \\
h_2      & h_1     & h_0     & 0      \\
0        & h_2     & h_1     & h_0    \\
0        & 0       & h_2     & h_1    \\
0        & 0       & 0       & h_2
\end{bmatrix}
\begin{bmatrix}
x_0\\
x_1\\
x_2\\
x_3
\end{bmatrix}$$

也就是：$y = T x$
其中 𝑇是 6×4 的 Toeplitz 矩阵。

**总结**
- 对长度为𝑁的信号 x 和长度为 𝐾 的卷积核 h，数学卷积 $y = x * h$可以写成：
$$y = T x$$
- 其中$y$长度为$N+K-1$, $T$是$(N+K-1) \times N$Toeplitz 矩阵
- Toeplitz 矩阵的第 1 列是$[h_0, h_1, \dots, h_{K-1}, 0, \dots, 0]^\top$，每一条斜对角线元素相同，对应“卷积核在信号上平移”的过程；
- 这就是“卷积 = 结构化矩阵乘法”的一个具体例子，还有
  - 用 GEMM 实现卷积（im2col）

  - 用 FFT / Circulant 做快速卷积

### 6.2.3 Circulant 矩阵

在周期边界条件下可变成 Circulant 矩阵，其特征：
定义：一个方阵 C 是 Circulant 矩阵，当且仅当每一行都是前一行的循环右移（或循环左移）。
例子：5×5 的循环矩阵
$$C = \begin{bmatrix}
c_0 & c_1 & c_2 & c_3 & c_4 \\
c_4 & c_0 & c_1 & c_2 & c_3 \\
c_3 & c_4 & c_0 & c_1 & c_2 \\
c_2 & c_3 & c_4 & c_0 & c_1 \\
c_1 & c_2 & c_3 & c_4 & c_0 \\
\end{bmatrix}$$

第一行是 [c₀ c₁ c₂ c₃ c₄]，后面每一行都是前一行向右循环移动一位，最后一列的元素回到最左边。

**circulant 矩阵性质**
| 序号 | 性质               | 数学描述                                                                                          | 对深度学习的意义                                                                 |
|------|------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| 1    | 可被傅里叶矩阵对角化   | $$ \mathbf{C} = \mathbf{F}^{-1} \operatorname{diag}(\operatorname{DFT}(\mathbf{c})) \mathbf{F} $$ <br>（F 是归一化的 DFT 矩阵） | 矩阵-向量乘法 Cx 只需 FFT → 点乘 → IFFT <br>→ 时间从 O(n²) 降到 O(n log n)，大核卷积和长序列 Transformer 必备加速技巧 |
| 2    | 乘法在傅里叶域退化为点乘 | $$ \mathbf{C}_1 \mathbf{x} = \operatorname{DFT}^{-1} \Big( \operatorname{DFT}(\mathbf{c}_1) \odot \operatorname{DFT}(\mathbf{x}) \Big) $$ | 循环卷积 ⇔ FFT 域逐元素相乘 <br>→ cuDNN / TensorFlow XLA / PyTorch 自动把大卷积转成 FFT 实现的核心理论 |
| 3    | Circulant 矩阵族对乘法封闭 | 若 C₁, C₂ 均为 Circulant 矩阵，<br>则 C₁C₂ 仍然是 Circulant 矩阵                                             | 多层堆叠后整体仍只需一个向量参数化 <br>→ 参数量指数级压缩，便于理论分析、谱范数估计、初始化与优化（CircConv、CKConv、S4 等模型的关键） |


**例子**
1. 直接用 FFT 实现大卷积（最常见加速技巧）
普通 1D 卷积：O(n⋅k)
用 FFT 实现的循环卷积：O(n log n)，当 k 很大时快几十倍。



### 6.2.4 总结
下面是深度学习、信号处理领域最常见的几种 Toeplitz 矩阵汇总表（直接复制即可完美显示）：

| 类型                        | 英文名称                        | 矩阵结构特点（以 5×5 为例）                                                                                          | 典型应用场景                                      | 备注                                      |
|-----------------------------|---------------------------------|-----------------------------------------------------------------------------------------------------------------------|--------------------------------------------------|------------------------------------------|
| 一般 Toeplitz                | Toeplitz                        | 每条从左上到右下的斜对角线元素相同                                                                                   | 标准一维线性卷积（valid/full 模式）                | 最常见的卷积矩阵形式                      |
| 循环 Toeplitz                | Circulant                       | 每行 = 前一行循环右移（或左移）<br>↓                                                                                   | FFT 快速卷积、周期信号处理、CircConv 压缩模型      | 可被 DFT 矩阵对角化，乘法超快             |
| 下三角 Toeplitz              | Lower Triangular Toeplitz       | 主对角线以下全部填满，对角线以上全 0<br>↓                                                                            | 因果滤波器、自回归模型（AR）、序列生成            | 只依赖当前及之前元素                     |
| 上三角 Toeplitz              | Upper Triangular Toeplitz       | 主对角线以上填满，以下全 0<br>↓                                                                                      | 反因果滤波器、反卷积、时间反转信号处理            | 只依赖当前及未来元素                      |
| Hankel                       | Hankel                          | 每条从左下到右上的反斜对角线元素相同（即每列是前一列向上循环移位）<br>↓                                       | 时刻序列分析、多项式乘法、系统辨识、Padé 逼近      | 和 Toeplitz 互为“转置+翻转”关系           |
| 循环 Hankel                  | Circulant Hankel                | 同时满足 Circulant + Hankel（极少见）                                                                                | 某些特殊周期平稳过程                              | 非常稀有                                  |
| 对称 Toeplitz（STS）          | Symmetric Toeplitz               | Toeplitz 且关于主对角线对称，即 t₋ᵢ = tᵢ                                                                               | 二阶平稳随机过程协方差矩阵、Wiener 滤波            | 实际信号统计中最常见                      |
| 块 Toeplitz（多通道）         | Block Toeplitz (Toeplitz-Block-Toeplitz) | 每个块本身是 Toeplitz，整个矩阵是大 Toeplitz                                                                          | 2D 卷积、多通道 1D 卷积的矩阵表示                  | CNN 完整卷积层权重矩阵就是这个            |

### 6.2.5 快速可视化示例（5×5）

| 类型               | 矩阵示例（用 t; c; h 表示不同值）                                      |
|--------------------|----------------------------------------------------------------------|
| Toeplitz           | t₀ t₁ t₂ t₃ t₄<br>t₋₁ t₀ t₁ t₂ t₃<br>t₋₂ t₋₁ t₀ t₁ t₂<br>...         |
| Circulant          | c₀ c₁ c₂ c₃ c₄<br>c₄ c₀ c₁ c₂ c₃<br>c₃ c₄ c₀ c₁ c₂<br>...           |
| Lower Triangular   | t₀ 0 0 0 0<br>t₋₁ t₀ 0 0 0<br>t₋₂ t₋₁ t₀ 0 0<br>...                |
| Upper Triangular   | 0 0 0 0 t₄<br>0 0 0 t₄ t₃<br>0 0 t₄ t₃ t₂<br>...                  |
| Hankel             | h₀ h₁ h₂ h₃ h₄<br>h₁ h₂ h₃ h₄ h₅<br>h₂ h₃ h₄ h₅ h₆<br>...         |
| Symmetric Toeplitz | t₀ t₁ t₂ t₃ t₄<br>t₁ t₀ t₁ t₂ t₃<br>t₂ t₁ t₀ t₁ t₂<br>...           |

这些矩阵几乎涵盖了你在 CNN、RNN、信号处理、时间序列领域能遇到的所有结构化线性变换！


**【工程视角】**  

- 理解 Toeplitz/Circulant 结构有助于：
  - 理解 FFT 卷积的本质
  - 理解某些“频域参数化”的卷积层
- 不需要记住复杂推导，但要知道：
  - 卷积可以被看成带特殊结构的矩阵
  - 特殊结构可以带来更快的乘法算法

---


In [7]:
### 6.2.3 用 FFT 实现线性卷积

import torch
import torch.nn.functional as F

x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8.], dtype=torch.float32)
h = torch.tensor([1, 2, 3.], dtype=torch.float32)

# 方法1：PyTorch conv1d（其实是 cross-correlation, valid）
out1 = F.conv1d(
    x.unsqueeze(0).unsqueeze(0),      # (1, 1, 8)
    h.unsqueeze(0).unsqueeze(0),      # (1, 1, 3)
    padding=0
).squeeze()                           # (6,)

# 方法2：FFT 实现同样的运算
N = x.size(0)
K = h.size(0)
n = N + K - 1                         # full 卷积长度

X = torch.fft.rfft(x, n=n)

# 为了得到 "correlation"，需要对 kernel 先翻转，再做卷积
h_flip = torch.flip(h, dims=[0])
Hf = torch.fft.rfft(h_flip, n=n)

y_full = torch.fft.irfft(X * Hf, n=n)  # full 线性卷积结果, 长度 n

# 从 full 里截出 valid 部分（和 conv1d 一样的 6 个点）
start = K - 1              # 2
end = start + (N - K + 1)  # 2 + 6 = 8
out2 = y_full[start:end]   # (6,)

print("conv1d 输出 out1:", out1)
print("FFT 复现 conv1d 的 out2:", out2)
print("是否一致:", torch.allclose(out1, out2, atol=1e-5))

conv1d 输出 out1: tensor([14., 20., 26., 32., 38., 44.])
FFT 复现 conv1d 的 out2: tensor([14., 20., 26., 32., 38., 44.])
是否一致: True



## 6.3 Winograd 卷积（最小乘法算法）
Winograd 来自数值分析与多项式插值理论，本质是：

> 用变换将局部卷积转成 element-wise 乘法，从而减少乘法次数。

### 6.3.1 为什么减少乘法重要？
在大多数硬件上：

- 乘法（特别是 FP16/INT8 的 MAC）比加法更贵

- 移动端 DSP / NPU 中更加明显（功耗限制）

Winograd 通过数学技巧：

- 把一个 tile 的卷积转成：

   - 输入变换

  - 卷积核变换

  - 元素级乘法

  - 输出逆变换

例如 F(2×2, 3×3) 算法：  
- 输入 tile 大小：4×4
- 输出 tile 大小：2×2
- 通过对输入和卷积核做线性变换，将卷积转化为：
  - element-wise 乘法
  - 再经过逆变换

$Y = A^\top \,\big[\, (G\,g\,G^\top)\;\;\;\; \odot\;\;\;\; (B^\top\, d \,B)\,\big] \; A$
- $d$： 输入 tile（例如 4×4）
- $g$: 卷积核（例如 3×3）
- $G, B, A$: 固定常数矩阵（transform matrices）
- $\odot$: Hadamard（逐元素）乘法
---
### 6.3.2 Winograd 公式的分解
1. 对输入进行 Winograd 变换
$$D = B^\top \, d\, B$$

- $d$ 是输入 tile，例如 4×4 patch
- $B$是固定常数矩阵（由 Winograd 理论推导）
- 得到经过变换的输入$D$
这一步只有 加法（+、–）、常数乘法
矩阵本身是固定常数 → 不需要 kernel 学习
所以它不是“真正的乘法 （FMA cost 很低）”
--- 

2. 对 kernel（权重）做同样的变换
$$G\, g\, G^\top$$

- kernel 为 3×3
- 𝐺 也是固定矩阵
- 得到变换后的 kernel
这一步可以在 offline（编译期）完成！
模型加载时只需 compute 一次 → 不计入推理时间
---

3. 逐元素相乘
$$M = (G g G^\top) \;\odot\; (B^\top d B)$$

winograd把卷积操作映射到一个「低阶多项式空间」在这个空间里卷积等价于多项式逐点相乘. 真正需要乘法的只有中间那 36 个 ⊙

---

4. 最后做一次逆变换得到输出 tile
$Y = A^\top\,M\,A$

- 𝐴 也是固定矩阵

- 只涉及加法、常数乘 → 很便宜

- 输出为 2×2 或 4×4 tile（取决于 F(m,r) 配置）
--- 

5. 为何 Winograd 乘法数量最少？
经典卷积（对 tile）乘法数量约为：

- 输入 tile 4×4
- kernel 3×3

直接卷积需要： 4×4 输入块 → 3×3 卷积核 → 得到 2×2 输出块（即 stride=1, no padding 的一个 tile）
- 普通 im2col + GEMM 4通道： $4\times 4\times 3\times 3 = 144 \text{ 次乘法}$  （Lavin 2016 《Fast Algorithms for Convolutional Neural Networks》）

- Winograd F(2×2, 3×3)：逐元素乘 4×4 = 16 次乘法（乘法减少 4×）

- Winograd F(4×4, 3×3)： 逐元素乘 6×6 = 36 次乘法，输出 tile 更大

二维 Winograd F(m×m, r×r)：
- 输出 tile： $m \times m$
- 卷积核： $r \times r$
- 中间Winograd 域的 tile 尺寸： $(m + r - 1) \times (m + r - 1)$
所以：
- 直接卷积乘法数（per tile, per in/out 通道对）， $\text{direct mults} = m^2 \cdot r^2$
- Winograd 乘法数（Hadamard 逐元素乘）$\text{winograd mults} = (m + r - 1)^2$


硬件上：

- 加法比乘法更便宜（特别在 DSP / NPU）

- 因此乘法减少 → 推理速度显著提升

| 场景（F(2×2, 3×3)）                                    | 乘法数           |
| ------------------------------------- | ------------- |
| **单通道真实卷积**                           | 36            |
| **单通道 im2col+GEMM**                   | 36            |
| **多输出通道 im2col+GEMM（C_out）**          | 9 × 4 × C_out |
| **C_out = 4 时**                       | **144**       |
| **Winograd Hadamard multiplies（单通道）** | 16            |
| **Winograd Hadamard multiplies（多通道）** | 16 × C_out    |

---
6. B、G、A 都是常数？

因为 Winograd minimal filtering 算法来自如下数学：

- 多项式插值

- Cook–Toom 槽分解

- 共轭变换

- 固定多项式基的分解

这些矩阵不依赖于
模型参数 / 输入内容 / 设备平台

所以：

- 训练不需要学
- 部署时不需要重新计算
- kernel 可以 offline-transform（一次即可
---

Winograd 其优点：

- 对小核卷积（如 3×3）乘法数大幅减少
- 在乘法远比加法“贵”的硬件上更有优势

缺点：

- 数值稳定性可能变差（放大误差）
  Winograd 的数学变换涉及：

   - 加法系数（如 1, -1, 1/2, -1/2, 2, ...）

    - 小数操作（非整数）

    - 数值放大与缩小（scaling）

    - 条件数变大（ill-conditioning）

  这会导致：

    - ❗ INT8 下严重的数值不稳定
    - overflow（溢出）
    - scale mismatch（量化尺度爆炸）  
    - 激活范围扩大
    - kernel transform 也需要 FP16/FP32 参与 很难保证 inference 结果稳定

  因此：

    > INT8 推理几乎都不用 Winograd Kernel。
    cuDNN / XNNPACK / QNNPACK / TensorRT / ARM Compute Library 全都明确这样做。

  而3×3 FP16/FP32 下 Winograd 数值稳定性足够 OK， FP16 硬件乘法很便宜， 可以减少 36 → 16 次乘法， 对 CPU/GPU 非常有效
- 对大核/stride 不友好
- 实现复杂

**【推理加速视角】**  

- 你需要知道：
  - 许多高性能 Conv kernel（尤其在移动端）使用 Winograd 或其变体
  - 它们都是在线性代数/多项式插值基础上得到的最小乘法算法
  - Winograd 卷积是 2015–2018 年移动端深度学习的救世主，但 2020 年后在高端 GPU 上被 Tensor Core 彻底打败，现在只活在老手机和嵌入式芯片里。
---

## 6.4 本章小结

1. 卷积定理：时域卷积 ↔ 频域乘法，为 FFT 卷积提供数学基础
2. Toeplitz/Circulant 矩阵提供了“卷积是结构化矩阵乘法”的视角
3. Winograd 算法通过巧妙变换减少乘法次数，是很多 Conv 加速方法的核心

在实际部署中，你不一定亲自实现 FFT/Winograd kernel，  
但理解其数学，有助于：

- 解读不同硬件/库对卷积的性能差异
- 做出“某些卷积形状适配某些算法”的合理判断。

【工程应用】

而是能做出正确判断：

1×1 → GEMM

3×3 → Winograd（FP16），Direct（INT8）直接卷积 + int8 量化算子

7×7/大 kernel → FFT

stride > 1 → Direct/GEMM

depthwise → specialized direct kernel

量化 → Direct/GEMM，不用 Winograd